In [1]:
import pandas as pd
import numpy as np

In [2]:
# from sqlalchemy import create_engine
import sqlite3

In [3]:
# Importation des données
df_population = pd.read_csv("data/fr_population.csv")
df_principale = pd.read_csv('data/df_prinicpale.csv')
df_sous_alimentation = pd.read_csv('data/df_sous_alimentation2013.csv')

# Traitement des dataframes pour faciliter leur utilisation par SQL

In [4]:
# df_population
df_population['Valeur'] = df_population['Valeur']*1000
df_population = df_population[df_population['Année']==2013][['Zone','Code zone','Année','Valeur']]
df_population.rename(columns={'Zone':'pays','Code zone':'code_pays','Année':'annee','Valeur':'population'},inplace = True)
df_population.to_csv("data_sql/population.csv", index = False)

In [5]:
# df_dispo_alim
df_principale['annee'] = 2013
df_principale['dispo_alim_tonnes'] = \
df_principale['Disponibilité alimentaire en quantité (kg/personne/an)'] * df_principale['Population']/1000

df_dispo_alim = df_principale[['Zone', 'Code zone', 'annee', 'Produit','Type',
                           'dispo_alim_tonnes', 'Disponibilité alimentaire (Kcal/personne/jour)',
                           'Disponibilité de protéines en quantité (g/personne/jour)',
                           'Disponibilité de matière grasse en quantité (g/personne/jour)']].copy()

df_dispo_alim.rename(columns = {'Zone':'pays', 'Code zone':'code_pays', 'Produit':'produit',
                                'Type':'origin', 'Disponibilité alimentaire (Kcal/personne/jour)':'dispo_alim_kcal_p_j',
                                'Disponibilité de protéines en quantité (g/personne/jour)':'dispo_prot',
                                'Disponibilité de matière grasse en quantité (g/personne/jour)':'dispo_mat_gr'}, inplace=True)                

df_dispo_alim.to_csv('data_sql/dispo_alim.csv', index = False)

In [6]:
#df_equilibre_prod
df_equilibre_prod = df_principale[['Zone', 'Code zone', 'annee', 'Produit', 'Disponibilité intérieure (kg/an)',
                             'Aliments pour animaux (kg/an)','Semences (kg/an)','Pertes (kg/an)',
                              'Traitement (kg/an)', 'Nourriture (kg/an)','Autres utilisations (non alimentaire) (kg/an)'
                              ]].copy()  
df_equilibre_prod.rename(columns = {'Zone':'pays', 'Code zone':'code_pays', 'Produit':'produit',
                                 'Disponibilité intérieure (kg/an)':'dispo_int', 
                                 'Aliments pour animaux (kg/an)':'alim_ani','Semences (kg/an)':'semences',
                                 'Pertes (kg/an)':'pertes', 'Traitement (kg/an)':'transfo','Nourriture (kg/an)':'nourriture',
                                 'Autres utilisations (non alimentaire) (kg/an)':'autres_utilisations'},inplace=True)

df_equilibre_prod.to_csv('data_sql/equilibre_prod.csv', index = False)

In [7]:
# df_sous_nutrition
# df_sous_alimentation2013['Année']=2013
#df_sous_nutrition = df_sous_alimentation.groupby(['Zone','Code zone']).sum().reset_index().copy()
df_sous_nutrition = df_sous_alimentation[['Zone', 'Code zone', 'Année', 'Valeur']].copy()
df_sous_nutrition.rename(columns={'Zone':'pays', 'Code zone':'code_pays', 'Année':'annee',
                                         'Valeur':'nb_personnes'}, inplace=True)
df_sous_nutrition['nb_personnes'] *= 1000000
df_sous_nutrition.to_csv('data_sql/sous_nutrition.csv', index = False)

<b> Connection à SQLite </b>

In [8]:
connection = sqlite3.connect("db_projet_1.db")

In [9]:
cursor = connection.cursor()

In [10]:
try:
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS population(
    pays TEXT(45) NOT NULL,
    code_pays INT, 
    annee INT, 
    population REAL,
    PRIMARY KEY(code_pays, annee)
    )
    ''')
    print('TABLE population créé avec succès')
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE population créé avec succès


In [11]:
try :
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS dispo_alim (
    pays TEXT(45) NOT NULL, 
    code_pays INT, 
    annee INT, 
    produit TEXT(45), 
    origin TEXT(20), 
    dispo_alim_tonnes REAL, 
    dispo_alim_kcal_p_j REAL, 
    dispo_prot REAL, 
    dispo_mat_gr REAL,
    PRIMARY KEY(code_pays,annee, produit)
    )
    ''')
    print('TABLE dispo_alim créé avec succès')
except sqlite3.Error as e:
    print("[Erreur] : ", e)
    

TABLE dispo_alim créé avec succès


In [12]:
try : 
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS equilibre_prod  (
    pays TEXT(45), 
    code_pays INT, 
    annee INT, 
    produit TEXT(45), 
    dispo_int REAL,
    alim_ani REAL, 
    semences REAL, 
    pertes REAL, 
    transfo REAL, 
    nourriture REAL, 
    autres_utilisations  REAL,
    PRIMARY KEY(code_pays,annee, produit)
    )
    ''')
    print('TABLE equilibre_prod créé avec succès')     
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE equilibre_prod créé avec succès


In [13]:
try:
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sous_nutrition (
    pays TEXT(45), 
    code_pays INT, 
    annee INT, 
    nb_personnes REAL,
    PRIMARY KEY(code_pays,annee)
    )
    ''')
    print('TABLE sous_nutrition créé avec succès')
    
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE sous_nutrition créé avec succès


In [14]:
try:
    df_population.to_sql('population', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [15]:
try :
    df_dispo_alim.to_sql('dispo_alim', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [16]:
try :
    df_equilibre_prod.to_sql('equilibre_prod', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [17]:
try:
    df_sous_nutrition.to_sql('sous_nutrition', con=connection, if_exists='append', index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [18]:
# cursor.execute("SELECT * FROM population").fetchall()

In [19]:
# connection.execute('INSERT INTO population VALUES ("Afghanistan", 2, 2013, 30552000)')

# Réponse 9 :


<b>Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.</b>

In [20]:
# cursor.execute('DROP VIEW view_ratio')

 pd.read_sql_query('''
CREATE VIEW view_ratio AS
SELECT pays, code_pays, annee, sum(dispo_prot) as ratio_proteine, sum(dispo_alim_kcal_p_j) as ratio_calorique
FROM dispo_alim
GROUP BY code_pays
''')


In [61]:
 pd.read_sql_query('''
SELECT pays
FROM view_ratio
ORDER BY ratio_proteine DESC
LIMIT 10
''',connection)

,pays
0,Islande
1,Israël
2,Lituanie
3,Maldives
4,Finlande
5,Luxembourg
6,Monténégro
7,Pays-Bas
8,Albanie
9,Portugal


<b>Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de kcal par habitant.</b>

In [62]:
 pd.read_sql_query('''
SELECT pays
FROM view_ratio
ORDER BY ratio_calorique DESC
LIMIT 10
''',connection)

,pays
0,Autriche
1,Belgique
2,Turquie
3,États-Unis d'Amérique
4,Israël
5,Irlande
6,Italie
7,Luxembourg
8,Égypte
9,Allemagne


<b>Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.
</b>

In [63]:
 pd.read_sql_query('''
SELECT pays
FROM view_ratio
WHERE annee=2013
ORDER BY ratio_proteine ASC
LIMIT 10
''',connection)

,pays
0,Libéria
1,Guinée-Bissau
2,Mozambique
3,République centrafricaine
4,Madagascar
5,Haïti
6,Zimbabwe
7,Congo
8,Ouganda
9,Sao Tomé-et-Principe


<b>La quantité totale (en kg) de produits perdus par pays en 2013.
</b>

In [64]:
 pd.read_sql_query('''
SELECT pays, sum(pertes) as quantite_produits_perdus
FROM equilibre_prod
WHERE annee = 2013
GROUP BY code_pays
''',connection)

,pays,quantite_produits_perdus
0,Arménie,2.280000e+08
1,Afghanistan,1.135000e+09
2,Albanie,2.760000e+08
3,Algérie,3.753000e+09
4,Angola,4.799000e+09
...,...,...
166,Luxembourg,1.100000e+07
167,Serbie,7.070000e+08
168,Monténégro,3.300000e+07
169,Soudan,1.670000e+08


<b>Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.
</b>

In [43]:
# SELECT *
# FROM population p, 
# RIGHT JOIN sous_nutrition sn on (sn.code_pays = p.code_pays and sn.annee= p.annee)

In [69]:
pays_sous_alimentation = cursor.execute('''
SELECT p.pays, sn.nb_personnes * 100 / p.population as proportion_sous_nutristion
FROM population p, sous_nutrition sn  
WHERE sn.code_pays = p.code_pays and sn.annee = p.annee
ORDER BY proportion_sous_nutristion DESC
LIMIT 10
''').fetchall()
# print('Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte :')
# for pays, proportion in pays_sous_alimentation:
#     print(pays)
pays_sous_alimentation

[('République centrafricaine', 51.99306759098787),
 ('Zimbabwe', 50.88339222614841),
 ('Haïti', 50.402248715711934),
 ('Zambie', 48.146364949446316),
 ('République populaire démocratique de Corée', 43.78389234786102),
 ('Congo', 42.71582733812949),
 ('Madagascar', 39.69465648854962),
 ('Libéria', 39.59012575687005),
 ('Tchad', 38.98635477582846),
 ('Ouganda', 36.19042550360574)]

In [67]:
pd.read_sql_query("SELECT pays, AVG(proportion) AS prop FROM (SELECT sous_nutrition.pays, nb_personnes / population AS proportion FROM sous_nutrition INNER JOIN population ON (sous_nutrition.annee = population.annee AND sous_nutrition.pays = population.pays) GROUP BY population.pays, population.annee) GROUP BY pays ORDER BY prop DESC LIMIT 10", connection)

,pays,prop
0,République centrafricaine,0.519931
1,Zimbabwe,0.508834
2,Haïti,0.504022
3,Zambie,0.481464
4,République populaire démocratique de Corée,0.437839
5,Congo,0.427158
6,Madagascar,0.396947
7,Libéria,0.395901
8,Tchad,0.389864
9,Ouganda,0.361904


<b>Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.</b>

In [60]:
 pd.read_sql_query('''
SELECT produit, 
       sum(autres_utilisations) / sum(dispo_int) AS ratio_autre
  FROM equilibre_prod
 GROUP BY produit
 ORDER BY ratio_autre DESC
 LIMIT 10;
''',connection)

,produit,ratio_autre
0,"Alcool, non Comestible",1.001288
1,Huil Plantes Oleif Autr,0.755024
2,Huile de Palmistes,0.704110
3,Huile de Palme,0.698109
4,Girofles,0.646154
5,Huile de Colza&Moutarde,0.551290
6,Graisses Animales Crue,0.470176
7,Huiles de Poissons,0.448609
8,Huile de Soja,0.416172
9,Plantes Aquatiques,0.379299


# Question 20 : pour quelques-uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur Internet !)

Le manioc est utilisé comme semoule ou comme fécule (tapioca)7,30 ou comme farine sans gluten31.

Les feuilles au-dessus de la plante peuvent être broyées pour fabriquer du pondu, un légume traditionnel.

Les plats les plus connus sont le foufou, l'attiéké un couscous de manioc, le Mpondu à base de manioc et de poisson, le pondu madesu, à base de manioc et de haricots.

Le manioc est aussi utilisé pour fabriquer une tortilla, le cassave, un pain la chikwangue et des bières traditionnelles7 telles la cachiri, le munkoyo ou la mbégé.

In [ ]:
cursor.execute('''


''').fetchall()

In [ ]:
connection.close()

In [52]:
df_principale[['Code zone','Produit','Pertes (kg/an)']].groupby("Produit").sum()

,Code zone,Pertes (kg/an)
Produit,,
Abats Comestible,21697,7.300000e+07
"Agrumes, Autres",19988,8.100000e+08
"Alcool, non Comestible",19983,0.000000e+00
Aliments pour enfants,21251,1.000000e+06
Ananas,20990,2.745000e+09
...,...,...
Viande de Suides,21577,2.750000e+08
Viande de Volailles,21697,3.850000e+08
"Viande, Autre",21697,1.300000e+07


In [56]:
df_equilibre_prod[['produit','pertes']].groupby("produit").sum()

,pertes
produit,
Abats Comestible,7.300000e+07
"Agrumes, Autres",8.100000e+08
"Alcool, non Comestible",0.000000e+00
Aliments pour enfants,1.000000e+06
Ananas,2.745000e+09
...,...
Viande de Suides,2.750000e+08
Viande de Volailles,3.850000e+08
"Viande, Autre",1.300000e+07
